# Packages

In [1]:
import cv2                                # state of the art computer vision algorithms library
import numpy as np                        # fundamental package for scientific computing
import mediapipe as mp                    # pose estimation through mp
import pyrealsense2 as rs                 # Intel RealSense cross-platform open-source API

# Setup Camera

In [2]:
# Initialize to retrieve the camera flow
pipe = rs.pipeline()
cfg = rs.config()

# Define the format of both stream
cfg.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

# Align both cameras
align = rs.align(rs.stream.color)

# Setup Calibration Target

In [4]:
# Define the parameters for cornerSubPix
winSize = (5, 5)  # Size of the window for searching sub-pixel corner
zeroZone = (-1, -1)  # No search zone restriction around the corner
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)  # Termination criteria

# Define the size of our calibration target
pattern_size = (7, 7)

objp = np.zeros((1, pattern_size[0] * pattern_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

images = []
images_corners = []

# Calibrated Stream

In [ ]:
# Start the capture
pipe.start(cfg)

while True:
    # Wait for a coherent pair of frames "depth and color" and align 
    # them
    frame = pipe.wait_for_frames()
    aligned_frame = align.process(frame)

    # Retrieve the color flow
    color_frame = frame.get_color_frame()
    
    if not color_frame:
        continue

    # Show the RGB frame
    color_image = np.asanyarray(color_frame.get_data())
    cv2.imshow('Color Image', color_image)

    # Find the target corner in the frame
    found, corners = cv2.findChessboardCorners(color_image, 
        pattern_size, flags=cv2.CALIB_CB_ADAPTIVE_THRESH + 
        cv2.CALIB_CB_FAST_CHECK +
        cv2.CALIB_CB_NORMALIZE_IMAGE)

    if cv2.waitKey(1) == ord('q'): break

    if found:
        print("Target found")
        # Draw the shape of the target
        images.append(objp)
        cv2.drawChessboardCorners(color_image, pattern_size, 
            corners, found)
        
        # Change RBG for Grayscale
        gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # Apply cornerSubPix on the gray picture
        corners_refined = cv2.cornerSubPix(gray_image, corners, 
            winSize, zeroZone, criteria)

        # Add the corner in the list
        images_corners.append(corners_refined)

        break

cv2.destroyAllWindows()
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(images, images_corners, gray_image.shape[::-1],None,None)

# Real size of the target (cm)
mire_size_cm = 23.3

# Focal in pixels accros the x-axis
fx = cameraMatrix[0, 0]  
taille_pixel_x = 1 / fx
# Focal in pixels accros the y-axis
fy = cameraMatrix[1, 1]  
taille_pixel_y = 1 / fy

decimation = rs.decimation_filter()
decimation.set_option(rs.option.filter_magnitude, 1)
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  # Start the capture
  #pipe.start(cfg)

  while True:
    # Wait for a coherent pair of frames "depth and color" and align them
    frame = pipe.wait_for_frames()
    aligned_frame = align.process(frame)

    # Retrieve the depth and color flow
    depth_frame = frame.get_depth_frame()
    color_frame = frame.get_color_frame()

    # Apply smothering filter
    spatial = rs.spatial_filter()
    spatial.set_option(rs.option.filter_magnitude, 5)
    spatial.set_option(rs.option.filter_smooth_alpha, 1)
    spatial.set_option(rs.option.filter_smooth_delta, 50)

    # Apply filters to fill the holes
    hole_filling = rs.hole_filling_filter()

    # Retrieve the images from both flow (colorized depth)
    colorizer = rs.colorizer()
    color_image = np.asanyarray(color_frame.get_data())

    decimated_depth = decimation.process(depth_frame)
    smoothed_depth = spatial.process(decimated_depth)
    filled_depth = hole_filling.process(smoothed_depth)
    depth_image = np.asanyarray(filled_depth.get_data())
    colorized_depth_frame = np.asanyarray(colorizer.colorize(filled_depth).get_data())

    # Add MP on the colored image
    color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
    color_image_rgb.flags.writeable = False

    results = pose.process(color_image_rgb)

    color_image_rgb.flags.writeable = True
    color_image_rgb = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)

    # Render detections
    mp_drawing.draw_landmarks(color_image_rgb, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                               )

    # Retrieve and print the coordinates for each landmark
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            x = int(landmark.x * color_image.shape[1])
            y = int(landmark.y * color_image.shape[0]) 
            # Print the calculated coordinates
            print(f"X = {x*taille_pixel_x}, Y = {y*taille_pixel_y}")
            
            # Ensure that the calculated pixel coordinates are within the bounds of the depth frame
            if 0 <= y < color_image.shape[0] and 0 <= x < color_image.shape[1]:
                depth_value = depth_frame.get_distance(x,y)
                print(f"Z = {depth_value}")
            else:
                print("Coordinates out of bounds")
            text = f"X={x*taille_pixel_x:.2f}, Y={y*taille_pixel_y:.2f}, Z={depth_value:.2f}"
            cv2.putText(color_image_rgb, text, (x + 10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)


    cv2.imshow("Depth", colorized_depth_frame)
    cv2.imshow("Color", color_image)
    cv2.imshow("MediaPipe", color_image_rgb)

    if cv2.waitKey(1) == ord('q'):
      break

  pipe.stop()
  cv2.destroyAllWindows()

Target found
